# Despliegue

In [2]:
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
#!pip install openpyxl
import openpyxl

import numpy as np

El despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
En este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.

In [3]:
#### conectarse a base de datos preprocesada

con=sql.connect('bases de datos RRHH/datos.db')
cur=con.cursor()

In [4]:
#crear dataframe para el modelado(2015)
df = pd.read_sql("""select * from completo
where InfoDate='2016-12-31'
""",con=con)

In [5]:
# Hacer la preparacion de los datos nuevos
df_t= fn.preparar_datos(df)

c:\Users\osiri\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# Cargar modelo entrenado
rf_final = joblib.load("salidas\\rf_final.pkl")

c:\Users\osiri\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\osiri\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Realizar predicciones
predicciones = rf_final.predict(df_t)
pd_pred = pd.DataFrame(predicciones, columns=['Atrition'])

# Crear DataFrame con predicciones
perf_pred = pd.concat([df['EmployeeID'], df_t, pd_pred], axis=1)

# Guardar predicciones en archivos
perf_pred[['EmployeeID', 'Atrition']].to_excel("salidas\\prediccion.xlsx")
#Guardar importancia de las caracteristicas a la hora de predecir
feature_names = df_t.columns


In [8]:
importances = pd.DataFrame({'Feature': feature_names, 'Importance': rf_final.feature_importances_})
importances.to_excel("salidas\\importances.xlsx")

In [9]:
# Ver las 10 predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=True).head(10)
emp_pred_bajo.set_index('EmployeeID', inplace=True)
pred = emp_pred_bajo.T
print(pred)

EmployeeID                      1      2800      2801      2802      2803  \
Age                      1.541369 -0.101159 -0.101159  0.665354 -0.758170   
EnvironmentSatisfaction  0.252200 -0.665460 -1.583120  0.252200 -1.583120   
MonthlyIncome            1.405136 -0.810167  2.350878 -0.870085 -0.783819   
NumCompaniesWorked      -0.677862 -1.078797  0.925878 -0.677862 -0.677862   
TotalWorkingYears       -1.320847 -0.163984 -0.549605  1.764122 -0.163984   
YearsAtCompany          -0.981014  0.325228 -0.981014  2.937711  0.488508   
Atrition                 0.000000  0.000000  0.000000  0.000000  0.000000   

EmployeeID                   2804      2806      2808      2811      2812  
Age                     -0.429664  1.541369 -1.196177 -1.634185  1.322365  
EnvironmentSatisfaction  0.252200 -1.583120  0.252200  0.252200  0.252200  
MonthlyIncome            1.007588  2.665772 -0.155945 -0.390946 -0.377985  
NumCompaniesWorked      -1.078797  1.727747 -0.677862 -1.078797 -0.677862  
Tot

In [12]:
import joblib
import warnings
from sklearn.preprocessing import StandardScaler

# Suprimir advertencias específicas de sklearn
warnings.filterwarnings("ignore", category=UserWarning)

# Cargar el scaler previamente guardado
scaler = joblib.load('salidas/scaler.pkl')

# Seleccionar las columnas escaladas
columnas_escaladas = ['Age', 'EnvironmentSatisfaction', 'MonthlyIncome', 'NumCompaniesWorked', 'TotalWorkingYears', 'YearsAtCompany']

# Extraer las posiciones de las columnas en el scaler original
columnas_idx = [list(scaler.feature_names_in_).index(col) for col in columnas_escaladas]

# Crear un nuevo scaler solo para las columnas seleccionadas
scaler_reducido = StandardScaler()
scaler_reducido.mean_ = scaler.mean_[columnas_idx]
scaler_reducido.scale_ = scaler.scale_[columnas_idx]
scaler_reducido.var_ = scaler.var_[columnas_idx]  # Incluyendo la varianza si es necesario

# Ordenar las predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=True).head(10)
emp_pred_bajo.set_index('EmployeeID', inplace=True)

# Seleccionar solo las columnas que fueron escaladas
df_pred_bajo_escalado = emp_pred_bajo[columnas_escaladas]

# Desescalar las predicciones
df_pred_bajo_original = pd.DataFrame(scaler_reducido.inverse_transform(df_pred_bajo_escalado), columns=df_pred_bajo_escalado.columns, index=df_pred_bajo_escalado.index)

# Concatenar las predicciones desescaladas con las columnas relevantes
df_pred_final = pd.concat([emp_pred_bajo[['Atrition']], df_pred_bajo_original], axis=1)

# Trasponer la tabla final
df_pred_final_traspuesta = df_pred_final.T

# Mostrar los resultados traspuestos
print(df_pred_final_traspuesta)


EmployeeID                      1     2800      2801     2802     2803  \
Atrition                      0.0      0.0       0.0      0.0      0.0   
Age                          51.0     36.0      36.0     43.0     30.0   
EnvironmentSatisfaction       3.0      2.0       1.0      3.0      1.0   
MonthlyIncome            131160.0  26900.0  175670.0  24080.0  28140.0   
NumCompaniesWorked            1.0      0.0       5.0      1.0      1.0   
TotalWorkingYears             1.0     10.0       7.0     25.0     10.0   
YearsAtCompany                1.0      9.0       1.0     25.0     10.0   

EmployeeID                   2804      2806     2808     2811     2812  
Atrition                      0.0       0.0      0.0      0.0      0.0  
Age                          33.0      51.0     26.0     22.0     49.0  
EnvironmentSatisfaction       3.0       1.0      3.0      3.0      3.0  
MonthlyIncome            112450.0  190490.0  57690.0  46630.0  47240.0  
NumCompaniesWorked            0.0       7.

* Edad: Los empleados predichos a renunciar presentan edades muy diversas, desde 22 hasta 51 años
Satisfacción con el ambiente laboral: La satisfacción laboral de los empleados varía entre valores bajos y moderados. El EmployeeID 2801 tiene una satisfacción laboral de 1, lo que sugiere una insatisfacción significativa.
Otros empleados, como el EmployeeID 1, tienen una satisfacción laboral de 3, lo que indica una satisfacción moderada, aunque no máxima.
Dado que ninguno de los empleados con predicción de renuncia tiene un valor de 4 en satisfacción laboral, podemos deducir que la insatisfacción o la satisfacción moderada está presente en estos empleados, lo que puede influir en su decisión de abandonar la empresa.
* Años en la empresa: Hay una diferencia considerable en los años trabajados en la empresa, desde 1 hasta 25 años. Los empleados que han estado en la empresa durante más tiempo, como el EmployeeID 2802 (25 años), también están en riesgo de renuncia, lo que podría deberse a factores como el agotamiento laboral o la falta de oportunidades de crecimiento.


Recomendaciones:
* Intervenir en la satisfacción laboral: Dado que varios de los empleados con riesgo de renuncia tienen una baja satisfacción con el ambiente laboral, se recomienda implementar programas para mejorar este aspecto, como fomentar una cultura laboral positiva y ofrecer canales de retroalimentación abierta.

* Revisión salarial: La variabilidad en los ingresos sugiere que podría haber una correlación entre los salarios más bajos y el riesgo de renuncia. Sería recomendable realizar una revisión salarial, especialmente para los empleados con ingresos bajos, para reducir el riesgo de pérdida de talento.

* Desarrollo y oportunidades de crecimiento: Los empleados que han trabajado en la empresa durante muchos años podrían estar enfrentando estancamiento profesional. Ofrecer oportunidades de desarrollo profesional y planes de carrera podría ayudar a retener a estos empleados.

* Monitorear a los empleados jóvenes y con movilidad laboral previa: Los empleados más jóvenes y aquellos que han trabajado en varias empresas parecen tener un mayor riesgo de rotación. Sería prudente ofrecerles más atención para aumentar su compromiso con la empresa.